In [ ]:
import os
import json
from pathlib import Path
import logging
import time
import random
from queue import Queue
import sys

from PIL import Image
from dotenv import load_dotenv

sys.path.append(os.path.join(os.path.dirname(__file__), '..'))

from gemini_api import GeminiAPIHandler

api_requests_file = "api_requests/api_requests_hiba.json"
output_dir = Path("api_results") / "hiba"
output_dir.mkdir(exist_ok=True)

log_filename = os.path.join(output_dir, f"api_resultscont.log")
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', 
    filename=log_filename
)
logger = logging.getLogger(__name__)

In [2]:
load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

model_name = "gemini-2.0-flash-exp"
#model_name = "gemini-2.0-flash-thinking-exp"

api_handler = GeminiAPIHandler(api_key=api_key, model_name=model_name)

with open(api_requests_file, "r") as file:
    api_requests = json.load(file)

In [3]:
def call_gemini_api(request: dict) -> dict:
    """
    Call the Gemini API for a given request and dataset image path.
    """
    image_id = request.get("image_id")
    image_path = request.get("image_path")
            
    # Get 2 prompts
    q1_general = request.get("q1_general")['prompt']
    q2_dataset_specific = request.get("q2_dataset_specific")['prompt']
    
    # Load the image
    with Image.open(image_path) as pil_image:
        # Generate response for general prompt
        q1_general_response = api_handler.generate_from_pil_image(pil_image, q1_general)
        # Generate response for dataset-specific prompt
        q2_dataset_response = api_handler.generate_from_pil_image(pil_image, q2_dataset_specific)
        
    # Save the responses
    result = {
        "image_id": image_id,
        "image_path": image_path,
        
        # Save the responses
        "q1_general_response": q1_general_response,
        "q2_dataset_response": q2_dataset_response,
        
        # Save the questions
        "q1_general_question": request.get("q1_general")['question'],
        "q2_dataset_question": request.get("q2_dataset_specific")['question'],
        
        # Save the prompts
        "q1_general_prompt": q1_general,
        "q2_dataset_prompt": q2_dataset_specific
    }
    
    return result

In [ ]:
requests_queue = Queue()
for req in api_requests:
    requests_queue.put(req)

results = []

while not requests_queue.empty():
    request = requests_queue.get()
    task_exists = False
    try:
        image_id = request.get("image_id")
        generated_response_file = output_dir / f"{image_id}_response.json"
        
        if os.path.exists(generated_response_file):
            logger.info(f"Skipping image_id {image_id} as response already exists.")
            task_exists = True

        else:
            result = call_gemini_api(request)
            results.append(result)
            with open(generated_response_file, "w") as out_file:
                json.dump(result, out_file, indent=4)
            logger.info(f"Successfully processed image_id {image_id}, saved to {generated_response_file}")

    except Exception as e:
        logger.warning(f"Error processing image_id {image_id}: {e}")
        
        # Requeue the request
        requests_queue.put(request)

    finally:
        requests_queue.task_done()
